In [ ]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array

import sys
sys.path.append('../..')
import snowFun

In [1]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "01_rgi60_Alaska", "01_rgi60_Alaska.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [2]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
print(rgis_to_analyze_df[:10])

3031
               RGIId     Area
548   RGI60-01.11632    [2.0]
2815  RGI60-01.20889    [2.0]
748   RGI60-01.25183  [2.002]
1388  RGI60-01.08203  [2.002]
473   RGI60-01.11595  [2.003]
1795  RGI60-01.09047  [2.004]
2069  RGI60-01.03614  [2.004]
2446  RGI60-01.05487  [2.004]
2212  RGI60-01.17509  [2.005]
2508  RGI60-01.10867  [2.005]


In [3]:
skip = 0
for i in range(len(rgis_to_analyze)):
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area']
    
    # define path that this is going to be saved to
    path_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks', f"S2_{rgiid}_mask.nc")
    
    # skip if it already exists
    if os.path.exists(path_save):
        continue
    
    # print progress
    print(f"Starting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga.values[0]} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # open the classification data
    file_name = f"S2_{rgiid}_2018-01-01_2023-01-01"
    xr_class = riox.open_rasterio(os.path.join(folder_class, f"{file_name}.tif")).chunk({'band':-1, 'y':1000, 'x':1000})#.rio.clip(single_geometry, from_disk=True, drop=True).chunk()

    # load metadata csv, convert date to datetimes
    meta_fp = os.path.join(folder_meta, f"{file_name}.csv")
    meta_df = pd.read_csv(meta_fp)
    
    # format time axis to pandas datetime, like xarray wants
    datetimes = pd.to_datetime([f"{str(i)[:4]}-{str(i)[4:6]}-{str(i)[6:]}" for i in meta_df['date']])
    xr_class = xr_class.rename({"band":"time"})
    xr_class['time'] = datetimes

    # merge images on same day, if there are repeated dates
#     if len(datetimes)!=len(datetimes.unique()):
#         xr_class = xr_class.where(xr_class<20, 0).groupby('time').max('time')
#     else:
#         xr_class = xr_class.where(xr_class<20, 0)
       
    # calculate maximum through time
    glacier_mask = xr_class.max(dim='time').rename('class')
    glacier_mask = (glacier_mask>0)#.astype(int)
    
    def format_xr_to_save(xr_da):
            xr_da.attrs["res"] = (10,10)
            xr_da.attrs["crs"] = "EPSG:3338"
            xr_da.attrs["transform"] = [10,0,0,0,-10,0]
#             xr_da.attrs["_FillValue"] = 0
            xr_da.attrs["long_name"] = rgiid
            xr_da.attrs["description"] = "0: off 1: on"
            xr_da.name = "glacier"

            xr_da.x.attrs["units"] = "meters"
            xr_da.y.attrs["units"] = "meters"
            xr_da.x.attrs["long_name"] = 'x'
            xr_da.y.attrs["long_name"] = 'y'

            return xr_da
        
    # format everything correctly    
    glacier_mask = format_xr_to_save(glacier_mask)
    
    # specify compression/encoding
    encoding = {"glacier":{"zlib": True}}#, "spatial_ref":{"zlib": False}}

    # save
    glacier_mask.to_netcdf(path_save, encoding=encoding)
        

Starting 3030 of 3031: RGI60-01.13635  3025.115 km2
